# Nanodegree Engenheiro de Machine Learning

## Projeto Final

### Fábio Corrêa Cordeiro
### Agosto 2018

# Escolhendo os parâmetros para melhorar o algoritmo

Nesta última etapa iremos gerar novamente os vetores dos documentos e treinar o algoritmo de classificação alterando as principais escolhas que fizemos até então. O objetivo é identificar a combinação de hiperparâmetros e algoritimos de classificação que retorne a maior métrica F1. Os parâmetros que serão testados são:
- Algoritmo de treinamento do Word2Vec: PV-DM ou PV-DBOW
- Dimensão do vetor: 25, 50, 100, 200 ou 400
- Janela de predição do algoritmo Word2Vec: 5, 10, 50, 100
- Épocas de iteração do algoritmo Word2Vec: 5, 10 ou 20
- Algoritmo de classificação: Decision Tree, Suport Vector Machine, Near Centroid, Gaussian Nayve Bayes, Random Forest e Neural Network.
- Redução de dimensionalidade: 0%, 50%, 80%, 90% ou 95%

Não será possível testar todas as combinações de parâmetro, seriam 3600 combinações diferentes. Portanto, para cada parâmetro, testaremos todas as opções mantendo o restante igual. Após encontrar os melhores valores para cada parâmentro faremos uma nova rodada testando cada parâmetros mantendo igual os demais. Após essas nova série de rodadas escolheremos a combinação com maior F1 score.

### Parâmetros iniciais

Esse notebook é referente à segunda rodada e os parâmetros que serão mantidos constantes são:
- Algoritmo de treinamento do Word2Vec: PV-DBOW
- Dimensão do vetor: 50
- Janela de predição do algoritmo Word2Vec: 5
- Épocas de iteração do algoritmo Word2Vec: 5
- Algoritmo de classificação: Near Centroid
- Redução de dimensionalidade: 80%

### Importando as bibliotecas necessárias

In [1]:
import numpy as np
import datetime
import pandas as pd
from collections import Counter
import os
from pathlib import Path
from random import shuffle
import re
import nltk
from  nltk.corpus import stopwords # Import the stop word list
from string import ascii_lowercase
import unicodedata
import gensim
from gensim.models.word2vec import PathLineSentences
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from sklearn.manifold import TSNE
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import train_test_split
from sklearn import tree
from sklearn.neighbors.nearest_centroid import NearestCentroid
from sklearn import svm
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import precision_recall_fscore_support, accuracy_score

C:\Users\fabio\Anaconda3\lib\site-packages\gensim\utils.py:1209: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


### Lendo os arquivos preprocessados

In [2]:
momentoInicial = datetime.datetime.now()   # Inicia um contador do tempo

pastaentrada = "corpus_preprocessado" # Pasta onde estão os arquivos de entrada
extensao = "*.txt"

# Iterando os arquivos da pasta de entrada
pastaArquivosCorpus = Path(pastaentrada).glob(extensao)

docs = []
arquivos = []

# Iterando os arquivos da pasta atual
for path in pastaArquivosCorpus:
    path_arquivo = str(path) # because path is object not string
    texto = ''
    with open(path_arquivo, 'r', encoding="UTF-8") as f:
        docs.append(gensim.utils.simple_preprocess(f.read()))
        arquivos.append(os.path.basename(path_arquivo))

momentoFinal = datetime.datetime.now() #Encerrando o contador do tempo
print("Tempo total decorrido: ", momentoFinal - momentoInicial)

Tempo total decorrido:  0:00:08.753958


### Função de para redução de dimensionalidade

In [3]:
def n_importante (n):
    classif = tree.DecisionTreeClassifier()
    classif = classif.fit(docvectors, label)
    
    # Reduzindo os vetores de 100 dimensões para as n mais importantes 
    max_index = np.argsort(classif.feature_importances_)[-n:]
    max_value = np.sort(classif.feature_importances_)[-n:]    # identificando 50 mais importante

    docvectors_n_features = []
    for vector in docvectors:
        docvectors_n_features.append(vector[max_index])         # criando uma variável com os 50 mais importante

    return (docvectors_n_features)

### Treinando o modelo de Doc2Vec e algoritmos de classificação 

Algoritmo de treinamento Word2Vec - PV-DM (dm = 1) ou **PV-DBOW (dm = 0)**

In [4]:
# dm ({1,0}, optional) – Defines the training algorithm.
# If dm=1, ‘distributed memory’ (PV-DM) is used.
# Otherwise, distributed bag of words (PV-DBOW) is employed.
param = [1, 0]

for p in param:
    print('Algoritmo de treinamento Word2Vec: dm =', p)
    
    momentoInicial = datetime.datetime.now()   # Inicia um contador do tempo

    # Lendo os documentos 
    documents = [TaggedDocument(doc, [i]) for i, doc in enumerate(docs)]
    # Gerando modelo Doc2Vec
    doc_model = Doc2Vec(documents, dm=p, vector_size=50, window=5, min_count=20, epochs=5, workers=3)

    # Separando uma lista com os vetores de cada documento
    docvectors = []

    n_vectors = len(range(len(doc_model.docvecs)))

    for n in range(n_vectors):
        docvectors.append(doc_model.docvecs[n])

    # Buscando os arquivos onde foi anotado o título e sub área
    titulo_label = pd.read_csv('lista_PRH_editada.csv')
    titulo = titulo_label['titulo'].tolist()     # transformando a coluna "titulo" em lista
    label = titulo_label['label'].tolist()       # transformando a coluna "label" em lista 

    docvectors = np.array(n_importante (int(50 * 0.2)))  # Reduzindo a dimensionalidade em 80%
    label = np.array(label)

    # Os dados serão divididos em 10 subconjuntos sendo 9 para treino e 1 para teste
    # Serão feitos 10 treinamentos e testes e a métrica final é a média fe todos o F1 score
    skf = StratifiedKFold(n_splits=10, shuffle=True)

    f1 = []
    acc = []
    precision = []
    recall = []

    for train_index, test_index in skf.split(docvectors, label):
        docvectors_train, docvectors_test = docvectors[train_index], docvectors[test_index]
        label_train, label_test = label[train_index], label[test_index]

        # treinando algoritmo decision tree e predizendo o conjunto de teste
        clf = NearestCentroid()
        clf = clf.fit(docvectors_train, label_train)
        label_pred = clf.predict(docvectors_test)

        # Definindo as métricas F1, Acurácia, Precisão e Revocação
        precision_n, recall_n, f1_n, support = precision_recall_fscore_support(label_test, label_pred, average='weighted')
        precision.append(precision_n)
        recall.append(recall_n)
        f1.append(f1_n)
        acc.append(accuracy_score(label_test, label_pred))

    print ('F1: ', np.mean(f1))

    momentoFinal = datetime.datetime.now() #Encerrando o contador do tempo
    print("Tempo total decorrido: ", momentoFinal - momentoInicial)
    print(" ")

Algoritmo de treinamento Word2Vec: dm = 1
F1:  0.6629651799692843
Tempo total decorrido:  0:00:18.598442
 
Algoritmo de treinamento Word2Vec: dm = 0
F1:  0.7383788164339883
Tempo total decorrido:  0:00:12.269041
 


Dimensão dos vetores: 25, 50, 100, **200** ou 400

In [5]:
# Dimensão dos vetores
param = [25, 50, 100, 200, 400]

for p in param:
    print('Dimensão dos vetores: ', p)
    
    momentoInicial = datetime.datetime.now()   # Inicia um contador do tempo

    # Lendo os documentos 
    documents = [TaggedDocument(doc, [i]) for i, doc in enumerate(docs)]
    # Gerando modelo Doc2Vec
    doc_model = Doc2Vec(documents, dm=0, vector_size=p, window=5, min_count=10, epochs=5, workers=3)

    # Separando uma lista com os vetores de cada documento
    docvectors = []

    n_vectors = len(range(len(doc_model.docvecs)))

    for n in range(n_vectors):
        docvectors.append(doc_model.docvecs[n])

    # Buscando os arquivos onde foi anotado o título e sub área
    titulo_label = pd.read_csv('lista_PRH_editada.csv')
    titulo = titulo_label['titulo'].tolist()     # transformando a coluna "titulo" em lista
    label = titulo_label['label'].tolist()       # transformando a coluna "label" em lista 

    docvectors = np.array(n_importante (int(p * 0.2)))  # Reduzindo a dimensionalidade em 80%
    label = np.array(label)

    # Os dados serão divididos em 10 subconjuntos sendo 9 para treino e 1 para teste
    # Serão feitos 10 treinamentos e testes e a métrica final é a média fe todos o F1 score
    skf = StratifiedKFold(n_splits=10, shuffle=True)

    f1 = []
    acc = []
    precision = []
    recall = []

    for train_index, test_index in skf.split(docvectors, label):
        docvectors_train, docvectors_test = docvectors[train_index], docvectors[test_index]
        label_train, label_test = label[train_index], label[test_index]

        # treinando algoritmo decision tree e predizendo o conjunto de teste        
        clf = NearestCentroid()
        clf = clf.fit(docvectors_train, label_train)
        label_pred = clf.predict(docvectors_test)

        # Definindo as métricas F1, Acurácia, Precisão e Revocação
        precision_n, recall_n, f1_n, support = precision_recall_fscore_support(label_test, label_pred, average='weighted')
        precision.append(precision_n)
        recall.append(recall_n)
        f1.append(f1_n)
        acc.append(accuracy_score(label_test, label_pred))

    print ('F1: ', np.mean(f1))

    momentoFinal = datetime.datetime.now() #Encerrando o contador do tempo
    print("Tempo total decorrido: ", momentoFinal - momentoInicial)
    print(" ")

Dimensão dos vetores:  25
F1:  0.5882478530531935
Tempo total decorrido:  0:00:14.188453
 
Dimensão dos vetores:  50
F1:  0.7319156524418747
Tempo total decorrido:  0:00:15.037516
 
Dimensão dos vetores:  100
F1:  0.6605521327193851
Tempo total decorrido:  0:00:16.374055
 
Dimensão dos vetores:  200
F1:  0.7418643014176054
Tempo total decorrido:  0:00:20.356920
 
Dimensão dos vetores:  400
F1:  0.71199471933662
Tempo total decorrido:  0:00:27.086079
 


Janela de predição do algoritmo Word2Vec: 5, 10, 50 ou **100**

In [6]:
# Janela de predição do algoritmo Word2Vec
param = [5, 10, 50, 100]

for p in param:
    print('Janela de predição: ', p)
    
    momentoInicial = datetime.datetime.now()   # Inicia um contador do tempo

    # Lendo os documentos 
    documents = [TaggedDocument(doc, [i]) for i, doc in enumerate(docs)]
    # Gerando modelo Doc2Vec
    doc_model = Doc2Vec(documents, dm=0, vector_size=50, window=p, min_count=10, epochs=5, workers=3)

    # Separando uma lista com os vetores de cada documento
    docvectors = []

    n_vectors = len(range(len(doc_model.docvecs)))

    for n in range(n_vectors):
        docvectors.append(doc_model.docvecs[n])

    # Buscando os arquivos onde foi anotado o título e sub área
    titulo_label = pd.read_csv('lista_PRH_editada.csv')
    titulo = titulo_label['titulo'].tolist()     # transformando a coluna "titulo" em lista
    label = titulo_label['label'].tolist()       # transformando a coluna "label" em lista 

    docvectors = np.array(n_importante (int(50 * 0.2)))  # Reduzindo a dimensionalidade em 80%
    label = np.array(label)

    # Os dados serão divididos em 10 subconjuntos sendo 9 para treino e 1 para teste
    # Serão feitos 10 treinamentos e testes e a métrica final é a média fe todos o F1 score
    skf = StratifiedKFold(n_splits=10, shuffle=True)

    f1 = []
    acc = []
    precision = []
    recall = []

    for train_index, test_index in skf.split(docvectors, label):
        docvectors_train, docvectors_test = docvectors[train_index], docvectors[test_index]
        label_train, label_test = label[train_index], label[test_index]

        # treinando algoritmo decision tree e predizendo o conjunto de teste
        clf = NearestCentroid()
        clf = clf.fit(docvectors_train, label_train)
        label_pred = clf.predict(docvectors_test)

        # Definindo as métricas F1, Acurácia, Precisão e Revocação
        precision_n, recall_n, f1_n, support = precision_recall_fscore_support(label_test, label_pred, average='weighted')
        precision.append(precision_n)
        recall.append(recall_n)
        f1.append(f1_n)
        acc.append(accuracy_score(label_test, label_pred))

    print ('F1: ', np.mean(f1))

    momentoFinal = datetime.datetime.now() #Encerrando o contador do tempo
    print("Tempo total decorrido: ", momentoFinal - momentoInicial)
    print(" ")

Janela de predição:  5
F1:  0.6701954769038182
Tempo total decorrido:  0:00:13.935596
 
Janela de predição:  10
F1:  0.6624728122016299
Tempo total decorrido:  0:00:12.950994
 
Janela de predição:  50
F1:  0.6282477145359915
Tempo total decorrido:  0:00:14.075879
 
Janela de predição:  100
F1:  0.6998051726610733
Tempo total decorrido:  0:00:14.215324
 


Épocas de iteração do algoritmo Word2Vec: 5, **10** ou 20

In [7]:
# Épocas de iteração do algoritmo Word2Vec
param = [5, 10, 20]

for p in param:
    print('Épocas de iteração do algoritmo Word2Vec: ', p)
    
    momentoInicial = datetime.datetime.now()   # Inicia um contador do tempo

    # Lendo os documentos 
    documents = [TaggedDocument(doc, [i]) for i, doc in enumerate(docs)]
    # Gerando modelo Doc2Vec
    doc_model = Doc2Vec(documents, dm=0, vector_size=50, window=5, min_count=10, epochs=p, workers=3)

    # Separando uma lista com os vetores de cada documento
    docvectors = []

    n_vectors = len(range(len(doc_model.docvecs)))

    for n in range(n_vectors):
        docvectors.append(doc_model.docvecs[n])

    # Buscando os arquivos onde foi anotado o título e sub área
    titulo_label = pd.read_csv('lista_PRH_editada.csv')
    titulo = titulo_label['titulo'].tolist()     # transformando a coluna "titulo" em lista
    label = titulo_label['label'].tolist()       # transformando a coluna "label" em lista 

    docvectors = np.array(n_importante (int(50 * 0.2)))  # Reduzindo a dimensionalidade em 80%
    label = np.array(label)

    # Os dados serão divididos em 10 subconjuntos sendo 9 para treino e 1 para teste
    # Serão feitos 10 treinamentos e testes e a métrica final é a média fe todos o F1 score
    skf = StratifiedKFold(n_splits=10, shuffle=True)

    f1 = []
    acc = []
    precision = []
    recall = []

    for train_index, test_index in skf.split(docvectors, label):
        docvectors_train, docvectors_test = docvectors[train_index], docvectors[test_index]
        label_train, label_test = label[train_index], label[test_index]

        # treinando algoritmo NearestCentroid e predizendo o conjunto de teste
        clf = NearestCentroid()
        clf = clf.fit(docvectors_train, label_train)
        label_pred = clf.predict(docvectors_test)

        # Definindo as métricas F1, Acurácia, Precisão e Revocação
        precision_n, recall_n, f1_n, support = precision_recall_fscore_support(label_test, label_pred, average='weighted')
        precision.append(precision_n)
        recall.append(recall_n)
        f1.append(f1_n)
        acc.append(accuracy_score(label_test, label_pred))

    print ('F1: ', np.mean(f1))

    momentoFinal = datetime.datetime.now() #Encerrando o contador do tempo
    print("Tempo total decorrido: ", momentoFinal - momentoInicial)
    print(" ")

Épocas de iteração do algoritmo Word2Vec:  5
F1:  0.6269334495967898
Tempo total decorrido:  0:00:15.491588
 
Épocas de iteração do algoritmo Word2Vec:  10
F1:  0.6625071484449857
Tempo total decorrido:  0:00:28.248928
 
Épocas de iteração do algoritmo Word2Vec:  20
F1:  0.6250393301712526
Tempo total decorrido:  0:00:53.899652
 


Algoritmo de classificação: Decision Tree, Suport Vector Machine, Near Centroid, **Gaussian Nayve Bayes**, Random Forest e Neural Network.

In [8]:
# Lendo os documentos 
documents = [TaggedDocument(doc, [i]) for i, doc in enumerate(docs)]
# Gerando modelo Doc2Vec
doc_model = Doc2Vec(documents, dm=0, vector_size=50, window=5, min_count=10, epochs=5, workers=3)

# Separando uma lista com os vetores de cada documento
docvectors = []

n_vectors = len(range(len(doc_model.docvecs)))

for n in range(n_vectors):
    docvectors.append(doc_model.docvecs[n])

# Buscando os arquivos onde foi anotado o título e sub área
titulo_label = pd.read_csv('lista_PRH_editada.csv')
titulo = titulo_label['titulo'].tolist()     # transformando a coluna "titulo" em lista
label = titulo_label['label'].tolist()       # transformando a coluna "label" em lista 

docvectors = np.array(n_importante (int(50 * 0.2)))  # Reduzindo a dimensionalidade em 80%
label = np.array(label)

In [9]:
# Algoritmo de classificação

dt = tree.DecisionTreeClassifier()
svc = svm.SVC(decision_function_shape='ovo')
nc = NearestCentroid()
gnb = GaussianNB()
rf = RandomForestClassifier(n_estimators=10, max_depth=None, min_samples_split=2, random_state=0)
nn = MLPClassifier(solver='lbfgs', alpha=1e-5, hidden_layer_sizes=(5, 2), random_state=1)

param = [dt, svc, nc, gnb, rf, nn]

for clf in param:
    
    momentoInicial = datetime.datetime.now()   # Inicia um contador do tempo
    
    print('Algoritmo de classificação: ', clf)
    
    # Os dados serão divididos em 10 subconjuntos sendo 9 para treino e 1 para teste
    # Serão feitos 10 treinamentos e testes e a métrica final é a média fe todos o F1 score    
    skf = StratifiedKFold(n_splits=10, shuffle=True)

    f1 = []
    acc = []
    precision = []
    recall = []

    for train_index, test_index in skf.split(docvectors, label):
        docvectors_train, docvectors_test = docvectors[train_index], docvectors[test_index]
        label_train, label_test = label[train_index], label[test_index]

        clf = clf.fit(docvectors_train, label_train)
        label_pred = clf.predict(docvectors_test)

        
        # Definindo as métricas F1, Acurácia, Precisão e Revocação
        precision_n, recall_n, f1_n, support = precision_recall_fscore_support(label_test, label_pred, average='weighted')
        precision.append(precision_n)
        recall.append(recall_n)
        f1.append(f1_n)
        acc.append(accuracy_score(label_test, label_pred))

    print ('F1: ', np.mean(f1))

    momentoFinal = datetime.datetime.now() #Encerrando o contador do tempo
    print("Tempo total decorrido: ", momentoFinal - momentoInicial)
    print(" ")

Algoritmo de classificação:  DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')
F1:  0.5430424992513786
Tempo total decorrido:  0:00:00.047988
 
Algoritmo de classificação:  SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovo', degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)
F1:  0.6653991265458501
Tempo total decorrido:  0:00:00.059985
 
Algoritmo de classificação:  NearestCentroid(metric='euclidean', shrink_threshold=None)
F1:  0.6274243286158713
Tempo total decorrido:  0:00:00.023994
 
Algoritmo de classificação:  GaussianNB(priors=None)
F1:  0.6697654532212617
Tempo t

C:\Users\fabio\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


F1:  0.6499501616760766
Tempo total decorrido:  0:00:00.251936
 
Algoritmo de classificação:  MLPClassifier(activation='relu', alpha=1e-05, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(5, 2), learning_rate='constant',
       learning_rate_init=0.001, max_iter=200, momentum=0.9,
       nesterovs_momentum=True, power_t=0.5, random_state=1, shuffle=True,
       solver='lbfgs', tol=0.0001, validation_fraction=0.1, verbose=False,
       warm_start=False)
F1:  0.5741215402343571
Tempo total decorrido:  0:00:01.436331
 


Redução de dimensionalidade: **0%**, 50%, 80%, 90% ou 95%

In [10]:
# Redução de dimensionalidade

docvectors                                   # sem redução
docvectors_50features = n_importante (50)    # redução de 50%
docvectors_20features = n_importante (20)    # redução de 80%
docvectors_10features = n_importante (10)    # redução de 90%
docvectors_5features = n_importante (5)      # redução de 95%

param = [[docvectors,'docvectors'],
         [docvectors_50features, 'docvectors_50features'],
         [docvectors_20features, 'docvectors_20features'],
         [docvectors_10features, 'docvectors_10features'],
         [docvectors_5features, 'docvectors_5features']]

for p in param:
    print('Redução de dimensionalidade: ', p[1])
    
    momentoInicial = datetime.datetime.now()   # Inicia um contador do tempo

    # Lendo os documentos 
    documents = [TaggedDocument(doc, [i]) for i, doc in enumerate(docs)]
    # Gerando modelo Doc2Vec
    doc_model = Doc2Vec(documents, vector_size=50, window=5, min_count=10, epochs=5, workers=3)

    # Separando uma lista com os vetores de cada documento
    docvectors = []

    n_vectors = len(range(len(doc_model.docvecs)))

    for n in range(n_vectors):
        docvectors.append(doc_model.docvecs[n])

    # Buscando os arquivos onde foi anotado o título e sub área
    titulo_label = pd.read_csv('lista_PRH_editada.csv')
    titulo = titulo_label['titulo'].tolist()     # transformando a coluna "titulo" em lista
    label = titulo_label['label'].tolist()       # transformando a coluna "label" em lista 

    docvectors = np.array(p[0])
    label = np.array(label)

    # Os dados serão divididos em 10 subconjuntos sendo 9 para treino e 1 para teste
    # Serão feitos 10 treinamentos e testes e a métrica final é a média fe todos o F1 score
    skf = StratifiedKFold(n_splits=10, shuffle=True)

    f1 = []
    acc = []
    precision = []
    recall = []

    for train_index, test_index in skf.split(docvectors, label):
        docvectors_train, docvectors_test = docvectors[train_index], docvectors[test_index]
        label_train, label_test = label[train_index], label[test_index]

        # treinando algoritmo NearestCentroid e predizendo o conjunto de teste
        clf = NearestCentroid()
        clf = clf.fit(docvectors_train, label_train)
        label_pred = clf.predict(docvectors_test)

        # Definindo as métricas F1, Acurácia, Precisão e Revocação
        precision_n, recall_n, f1_n, support = precision_recall_fscore_support(label_test, label_pred, average='weighted')
        precision.append(precision_n)
        recall.append(recall_n)
        f1.append(f1_n)
        acc.append(accuracy_score(label_test, label_pred))

    print ('F1: ', np.mean(f1))

    momentoFinal = datetime.datetime.now() #Encerrando o contador do tempo
    print("Tempo total decorrido: ", momentoFinal - momentoInicial)
    print(" ")

Redução de dimensionalidade:  docvectors
F1:  0.6356753616214961
Tempo total decorrido:  0:00:19.641465
 
Redução de dimensionalidade:  docvectors_50features
F1:  0.6197263053424058
Tempo total decorrido:  0:00:19.810776
 
Redução de dimensionalidade:  docvectors_20features
F1:  0.615006843233204
Tempo total decorrido:  0:00:19.317378
 
Redução de dimensionalidade:  docvectors_10features
F1:  0.6206736802415193
Tempo total decorrido:  0:00:19.755510
 
Redução de dimensionalidade:  docvectors_5features
F1:  0.573279984083611
Tempo total decorrido:  0:00:18.560741
 
